In [1]:
import boto3
import botocore

import json

In [2]:
# AWS Clients

braket_config = botocore.client.Config(user_agent_extra="BraketSchemas/1.8.0")

aws_braket = boto3.client("braket", config=braket_config)
aws_s3 = boto3.client("s3")

In [3]:
# Load info for Tasks simulated on SV1

SV1_ARN = 'arn:aws:braket:::device/quantum-simulator/amazon/sv1'

response = aws_braket.search_quantum_tasks(filters=[{
    'name': 'deviceArn',
    'operator': 'EQUAL',
    'values': [SV1_ARN]
}], maxResults=25)

In [4]:
quantum_tasks = response['quantumTasks']

quantum_task = quantum_tasks[0]

quantum_task

{'createdAt': datetime.datetime(2023, 2, 28, 16, 55, 2, 235000, tzinfo=tzutc()),
 'deviceArn': 'arn:aws:braket:::device/quantum-simulator/amazon/sv1',
 'endedAt': datetime.datetime(2023, 2, 28, 16, 55, 11, 962000, tzinfo=tzutc()),
 'outputS3Bucket': 'amazon-braket-us-east-1-196332284574',
 'outputS3Directory': 'tasks/9581aea9-5357-4dba-bf3e-f97e19d5750e',
 'quantumTaskArn': 'arn:aws:braket:us-east-1:196332284574:quantum-task/9581aea9-5357-4dba-bf3e-f97e19d5750e',
 'shots': 1000,
 'status': 'COMPLETED',
 'tags': {}}

In [5]:
s3_bucket_name = quantum_task['outputS3Bucket']
s3_directory = quantum_task['outputS3Directory']
    
s3_bucket_name, s3_directory

('amazon-braket-us-east-1-196332284574',
 'tasks/9581aea9-5357-4dba-bf3e-f97e19d5750e')

In [6]:
# Download JSON

RESULTS_JSON_FILENAME = 'results-downloaded.json'

aws_s3.download_file(Bucket=s3_bucket_name,
                     Key=s3_directory + '/results.json',
                     Filename=RESULTS_JSON_FILENAME)

In [7]:
with open(RESULTS_JSON_FILENAME) as json_file:
    
    result_dict = json.load(json_file)
    

In [8]:
# result_dict['measurements'][0]

In [40]:
qasm_string = result_dict['additionalMetadata']['action']['source']

# print(QASM_string)

In [53]:
def print_qasm(qasm_string):
    
    qasm_lines = qasm_string.splitlines()
    
    for line_number, line in enumerate(qasm_lines, 1):
    
        print(f"{line_number:2<}    {line}")

In [54]:
from qiskit import qasm3, QuantumCircuit

import qiskit.qasm3

In [70]:
qc = QuantumCircuit(2,1)
qc.h(0)
qc.cx(0, 1)
qc.measure(0,0)
# qc.x(1).c_if(0, 0)


example_qasm_string = qasm3.dumps(qc)

In [71]:
print_qasm(example_qasm_string)

1    OPENQASM 3;
2    include "stdgates.inc";
3    bit[1] c;
4    qubit[2] _all_qubits;
5    let q = _all_qubits[0:1];
6    h q[0];
7    cx q[0], q[1];
8    c[0] = measure q[0];


In [72]:
circuit = qasm3.loads(example_qasm_string)

circuit.draw()

┌───┐     ┌─┐
esc__all_qubits_0: ┤ H ├──■──┤M├
                   └───┘┌─┴─┐└╥┘
esc__all_qubits_1: ─────┤ X ├─╫─
                        └───┘ ║ 
              c: 1/═══════════╩═
                              0

In [73]:
print_qasm(qasm_string)

1    OPENQASM 3.0;
2    qubit[28] q;
3    ry(0.0) q[0];
4    ry(0.0) q[1];
5    ry(0.0) q[2];
6    ry(0.0) q[3];
7    ry(0.0) q[4];
8    ry(0.0) q[5];
9    ry(0.0) q[6];
10    ry(0.0) q[7];
11    ry(0.0) q[8];
12    ry(0.0) q[9];
13    ry(0.0) q[10];
14    ry(0.0) q[11];
15    ry(0.0) q[12];
16    ry(0.0) q[13];
17    ry(0.0) q[14];
18    ry(0.0) q[15];
19    ry(0.0) q[16];
20    ry(0.0) q[17];
21    ry(0.0) q[18];
22    ry(0.0) q[19];
23    ry(0.0) q[20];
24    ry(0.0) q[21];
25    ry(0.0) q[22];
26    ry(0.0) q[23];
27    ry(0.0) q[24];
28    ry(0.0) q[25];
29    ry(0.0) q[26];
30    ry(0.0) q[27];
31    rx(0.8756000370805986) q[27];
32    cnot q[0], q[5];
33    cnot q[4], q[25];
34    rx(5.246021396377329) q[18];
35    cnot q[10], q[23];
36    cnot q[0], q[15];
37    ry(6.185181318152103) q[14];
38    ry(3.3030088295727937) q[7];
39    ry(1.0786927663891448) q[20];
40    cnot q[18], q[20];
41    ry(1.710957393023853) q[5];
42    cnot q[20], q[6];
43    rz(0.11555202867213518) q[4];


In [74]:
circuit = qasm3.loads(qasm_string)

circuit.draw()

QASM3ImporterError: "3,0: gate 'ry' is not defined."

### AWS S3 Examples

In [27]:
aws_s3.upload_file(Filename='test.txt', 
                   Bucket='test-braket-bucket', 
                   Key='test-remote.txt')

In [28]:
aws_s3.download_file(Bucket='test-braket-bucket',
                     Key='test-remote.txt',
                     Filename='test-downloaded.txt')

In [30]:
with open('test-downloaded.txt') as file:

    print(file.read())

Test Task
